In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("train.csv")
df.head()

,text,sentiment
0,"oh Marly, I`m so sorry!! I hope you find her...",neutral
1,Playing Ghost Online is really interesting. Th...,positive
2,is cleaning the house for her family who is co...,neutral
3,gotta restart my computer .. I thought Win7 wa...,neutral
4,SEe waT I Mean bOuT FoLL0w fRiiDaYs... It`S cA...,neutral


In [2]:
df.describe()

,text,sentiment
count,27447,27448
unique,27410,3
top,#NAME?,neutral
freq,38,11106


In [3]:

df.dropna(axis = 0,inplace = True)
df.shape

(27447, 2)

In [4]:
#removal of stop words
import nltk #nlp library
#nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words("english")
df['text'] = df['text'].apply(lambda x: " ".join([word for word in x.split() if word not in (stop)]))
df.head()

,text,sentiment
0,"oh Marly, I`m sorry!! I hope find soon!! <3 <3",neutral
1,Playing Ghost Online really interesting. The n...,positive
2,cleaning house family comming later today..,neutral
3,gotta restart computer .. I thought Win7 suppo...,neutral
4,SEe waT I Mean bOuT FoLL0w fRiiDaYs... It`S cA...,neutral


In [5]:
#Stemming
from nltk.stem import PorterStemmer
#nltk.download('punkt')
ps = PorterStemmer()
df['text'] = df['text'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))
df.head()


,text,sentiment
0,"oh marly, i`m sorry!! I hope find soon!! <3 <3",neutral
1,play ghost onlin realli interesting. the new u...,positive
2,clean hous famili com later today..,neutral
3,gotta restart comput .. I thought win7 suppos ...,neutral
4,see wat I mean bout foll0w friidays... it` cal...,neutral


In [7]:
#Lemmatization
from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
lemma = WordNetLemmatizer()
df['text'] = df['text'].apply(lambda x : ' '.join([lemma.lemmatize(word) for word in x.split()]))
df.head()

,text,sentiment
0,"oh marly, i`m sorry!! I hope find soon!! <3 <3",neutral
1,play ghost onlin realli interesting. the new u...,positive
2,clean hous famili com later today..,neutral
3,gotta restart comput .. I thought win7 suppos ...,neutral
4,see wat I mean bout foll0w friidays... it` cal...,neutral


In [8]:
#removing punctuation
import re
def remove_punct(text):
    text = re.sub('http://[A-Za-z0-9./]+','',text) #removes any https links
    text = re.sub('[0-9]', '',text) #removes numbers
    text = re.sub("[^a-zA-Z0-9]", " ",text) #removes special characters
    return text
df['text'] = df['text'].apply(remove_punct)
df.head()

,text,sentiment
0,oh marly i m sorry I hope find soon,neutral
1,play ghost onlin realli interesting the new u...,positive
2,clean hous famili com later today,neutral
3,gotta restart comput I thought win suppos p...,neutral
4,see wat I mean bout follw friidays it call...,neutral


In [9]:

#Spell correction
from textblob import TextBlob
df['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0          oh mary  i m sorry   I hope find soon      
1    play ghost online really interesting  the new ...
2                 clean house family com later today  
3    gutta start compute    I thought win suppose p...
4    see wat I mean bout follow friday    it  call ...
Name: text, dtype: object

In [10]:

#capitalizing the tweets
df['text'] = df['text'].apply(lambda x: " ".join([word.upper() for word in x.split()]))
df['text'].head()

0                  OH MARLY I M SORRY I HOPE FIND SOON
1    PLAY GHOST ONLIN REALLI INTERESTING THE NEW UP...
2                    CLEAN HOUS FAMILI COM LATER TODAY
3    GOTTA RESTART COMPUT I THOUGHT WIN SUPPOS PUT ...
4    SEE WAT I MEAN BOUT FOLLW FRIIDAYS IT CALL LOS...
Name: text, dtype: object

In [11]:

#tokenizing

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
def token(text):
    return tknzr.tokenize(text)
df['tokenized'] = df['text'].apply(token) 
df.head()

#method 2
#def tokenization(text):
#    return text.split()

#df['Tweet_tokenized'] = df['text'].apply(tokenization)
#df.head()

,text,sentiment,tokenized
0,OH MARLY I M SORRY I HOPE FIND SOON,neutral,"[OH, MARLY, I, M, SORRY, I, HOPE, FIND, SOON]"
1,PLAY GHOST ONLIN REALLI INTERESTING THE NEW UP...,positive,"[PLAY, GHOST, ONLIN, REALLI, INTERESTING, THE,..."
2,CLEAN HOUS FAMILI COM LATER TODAY,neutral,"[CLEAN, HOUS, FAMILI, COM, LATER, TODAY]"
3,GOTTA RESTART COMPUT I THOUGHT WIN SUPPOS PUT ...,neutral,"[GOTTA, RESTART, COMPUT, I, THOUGHT, WIN, SUPP..."
4,SEE WAT I MEAN BOUT FOLLW FRIIDAYS IT CALL LOS...,neutral,"[SEE, WAT, I, MEAN, BOUT, FOLLW, FRIIDAYS, IT,..."


In [12]:
#label encoding
df['sentiment'] = df['sentiment'].map({'neutral':0,'positive':1,'negative':2})

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=5000, max_df=0.7)  
x = tfidfconverter.fit_transform(df['text']).toarray() #this is x(used to train the model)that has text converted to vectors

In [14]:
from sklearn.model_selection import train_test_split #for splitting the data
from sklearn import metrics
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

y = df['sentiment']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=101)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(max_iter = 1000000, solver='saga')
#scores = cross_val_score(lr, x_train,y_train, cv=10)
logreg = lr.fit(x_train,y_train)
prediction = logreg.predict(x_test)
metrics.accuracy_score(y_test,prediction)

0.6985428051001822

In [15]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.64      0.76      0.69      2247
           1       0.79      0.69      0.74      1773
           2       0.70      0.62      0.66      1470

    accuracy                           0.70      5490
   macro avg       0.71      0.69      0.70      5490
weighted avg       0.71      0.70      0.70      5490



In [17]:
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import TimeDistributed

Using TensorFlow backend.


In [18]:
from sklearn.model_selection import train_test_split
text = df['text'].values
labels = df['sentiment'].values
text_train, text_test, y_train, y_test = train_test_split(text, labels)
print(text_train.shape, text_test.shape, y_train.shape, y_test.shape)

(20585,) (6862,) (20585,) (6862,)


In [24]:
vocab_size = 20000
maxlen = 5000
embedding_size = 32

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(text)

X_train = tokenizer.texts_to_sequences(text_train)
x_test = tokenizer.texts_to_sequences(text_test)

X_train = pad_sequences(X_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [25]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=maxlen))
model.add(LSTM(128))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(1, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5000, 32)          640000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 722,561
Trainable params: 722,561
Non-trainable params: 0
_________________________________________________________________


In [26]:
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        min_delta=1e-4,
        patience=3,
        verbose=1
    ),
    ModelCheckpoint(
        filepath='weights.h5',
        monitor='val_accuracy', 
        mode='max', 
        save_best_only=True,
        save_weights_only=True,
        verbose=1
    )
]

In [16]:
vds = pd.read_csv("validation.csv")
vds.head()

,text
0,Last session of the day http://twitpic.com/67ezh
1,Shanghai is also really exciting (precisely -...
2,"Recession hit Veronique Branquinho, she has to..."
3,happy bday!
4,http://twitpic.com/4w75p - I like it!!


In [17]:

vds.shape

(3534, 1)

In [18]:
vds.dropna(axis = 0,inplace = True)
vds.shape
#hence no null values

(3534, 1)

In [19]:
#removal of stop words
import nltk #nlp library
from nltk.corpus import stopwords
stop = stopwords.words("english")
vds['text'] = vds['text'].apply(lambda x: " ".join([word for word in x.split() if word not in (stop)]))
vds.head()

,text
0,Last session day http://twitpic.com/67ezh
1,Shanghai also really exciting (precisely -- sk...
2,"Recession hit Veronique Branquinho, quit compa..."
3,happy bday!
4,http://twitpic.com/4w75p - I like it!!


In [20]:

#Stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
vds['text'] = vds['text'].apply(lambda x : ' '.join([ps.stem(word) for word in x.split()]))
vds.head()


,text
0,last session day http://twitpic.com/67ezh
1,shanghai also realli excit (precis -- skyscrap...
2,"recess hit veroniqu branquinho, quit company, ..."
3,happi bday!
4,http://twitpic.com/4w75p - I like it!!


In [21]:
#Lemmatization
from nltk.stem import WordNetLemmatizer
lemma = WordNetLemmatizer()
vds['text'] = vds['text'].apply(lambda x : ' '.join([lemma.lemmatize(word) for word in x.split()]))
vds.head()

,text
0,last session day http://twitpic.com/67ezh
1,shanghai also realli excit (precis -- skyscrap...
2,"recess hit veroniqu branquinho, quit company, ..."
3,happi bday!
4,http://twitpic.com/4w75p - I like it!!


In [22]:
#removing punctuation
import re
def remove_punct(text):
    text = re.sub('http://[A-Za-z0-9./]+','',text) #removes any https links
    text = re.sub('[0-9]', '',text) #removes numbers
    text = re.sub("[^a-zA-Z0-9]", " ",text) #removes special characters
    return text
vds['text'] = vds['text'].apply(remove_punct)
vds.head()

,text
0,last session day
1,shanghai also realli excit precis skyscrap...
2,recess hit veroniqu branquinho quit company ...
3,happi bday
4,I like it


In [23]:
#Spell correction
from textblob import TextBlob
vds['text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0                                    last session day 
1    shanghai also really exit  precise    skyscrap...
2    recess hit veroniqu branquinho  quit company  ...
3                                           happy day 
4                                          I like it  
Name: text, dtype: object

In [24]:
#tokenizing
def tokenization(text):
    return text.split()

vds['Tweet_tokenized'] = vds['text'].apply(tokenization)
vds.head()

,text,Tweet_tokenized
0,last session day,"[last, session, day]"
1,shanghai also realli excit precis skyscrap...,"[shanghai, also, realli, excit, precis, skyscr..."
2,recess hit veroniqu branquinho quit company ...,"[recess, hit, veroniqu, branquinho, quit, comp..."
3,happi bday,"[happi, bday]"
4,I like it,"[I, like, it]"


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=5000, max_df=0.7)  
x_valid = tfidfconverter.fit_transform(vds['text']).toarray()

In [26]:

import pickle #basically should contain all the trained models that can be used again( to avoid reloading of models)
  
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(lr) 
  
# Load the pickled model 
lr_from_pickle = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions 
y_pred = lr_from_pickle.predict(x_valid) 
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [27]:
sentiment1 = lr_from_pickle.predict(x_test) 
sentiment1

array([0, 0, 2, ..., 0, 0, 0], dtype=int64)

In [28]:

vds['sentiment']=pd.Series(y_pred)
#del vds['Tweet_tokenized']
vds.head()

,text,Tweet_tokenized,sentiment
0,last session day,"[last, session, day]",0
1,shanghai also realli excit precis skyscrap...,"[shanghai, also, realli, excit, precis, skyscr...",0
2,recess hit veroniqu branquinho quit company ...,"[recess, hit, veroniqu, branquinho, quit, comp...",0
3,happi bday,"[happi, bday]",0
4,I like it,"[I, like, it]",0


In [29]:

vds.sentiment[vds.sentiment == 1] = 'positive'
vds.sentiment[vds.sentiment == 0] = 'neutral'
vds.sentiment[vds.sentiment == 2] = 'negative'
vds['sentiment']
vds.to_csv('logistic_regression.csv')